In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import json
import numpy as np


In [2]:
df = pd.read_csv("Karşılaştırma.csv")
df = df[df.columns[1:]]

## 2 modelinde tahmin edemediklerini çıkar ve yazdır

In [3]:
df[df["LLM tahmin"].isna() & df["VLLM tahmin"].isna()]

,username,creator_id,LLM tahmin,VLLM tahmin,LLM reasoning,VLLM reasoning
0,flyiiingtwins,711,NaN,NaN,Insufficient evidence to determine nationality.,No visual or textual indicators of nationality...
1,chillestcat,716,NaN,NaN,Insufficient evidence to determine nationality.,No visual or textual evidence indicates nation...
2,ringoantonio,722,NaN,NaN,Insufficient evidence to determine nationality.,Insufficient visual or textual evidence to det...
3,lillthicky,723,NaN,NaN,Insufficient evidence to determine nationality.,Profile image is a generic emoji with no cultu...
5,thesierracapri,727,NaN,NaN,Insufficient evidence to determine nationality.,Profile image shows a person with features tha...
...,...,...,...,...,...,...
212,yamiso.her,1345,NaN,NaN,Insufficient evidence to determine nationality.,No visual or textual indicators of nationality...
214,beautybycindyllc,1357,NaN,NaN,Insufficient evidence to determine nationality.,Profile image shows ambiguous ethnicity; text ...
217,kristy.sarah,1365,NaN,NaN,Insufficient evidence to determine nationality.,No clear visual or textual indicators of natio...
218,bigmog24,1375,NaN,NaN,Insufficient information to determine national...,No clear textual or visual indicators of natio...


## En az 1 modelin tahmin ettiği değerler

In [4]:
df = df[df["LLM tahmin"].notna() | df["VLLM tahmin"].notna()]
df

,username,creator_id,LLM tahmin,VLLM tahmin,LLM reasoning,VLLM reasoning
4,javirod305,726,United States,American,"Bio mentions Miami and NYC, indicating U.S. re...",Bio states 'A Miami boy living in NYC' and use...
12,queenisemhe1,747,NaN,Nigerian,Insufficient evidence to determine nationality.,The couple is wearing traditional Nigerian att...
16,dleysafricanhairbraiding,757,United States,American,"Bio mentions Camden, New Jersey, USA","Address 'Camden Nj' (New Jersey, USA) in bio s..."
22,calintee38,776,Canada,Canadian,Bio explicitly states 'Canadian creator',Bio explicitly states 'Canadian creator'.
25,labellegrachi,779,France,French,Bio and post text in French,High confidence in French language detection (...
...,...,...,...,...,...,...
209,miahhcarter,1335,United Kingdom,United Kingdom,Email domain indicates UK management company,Email domain 'eminencemanagement.co.uk' indica...
211,jxj03l,1337,NaN,Netherlands,Insufficient evidence to determine nationality.,"Language detected is Dutch, strongly indicatin..."
213,juliusburphy,1355,United Kingdom,United Kingdom,Bio flag and content suggest UK origin,Bio explicitly includes the UK flag emoji (🇬🇧)...
215,myla.rnd,1358,France,French,"Text in French, no contradicting evidence.",The French phrase 'je voulais voir s’il regard...


### LLM modelinin tahmin edemeyip VLM modelinin tahmin edebildiği değerler

In [5]:
a = df[df["LLM tahmin"].isna() & df["VLLM tahmin"].notna()].shape[0]
print(f"LLM modelinin tahmin edemeyip VLLM Modelinin tahmin edebildiğiği toplam değer: {a}")
df[df["LLM tahmin"].isna() & df["VLLM tahmin"].notna()]

LLM modelinin tahmin edemeyip VLLM Modelinin tahmin edebildiğiği toplam değer: 7


,username,creator_id,LLM tahmin,VLLM tahmin,LLM reasoning,VLLM reasoning
12,queenisemhe1,747,NaN,Nigerian,Insufficient evidence to determine nationality.,The couple is wearing traditional Nigerian att...
83,user11004336799174,925,NaN,Chinese,No identifiable information provided.,The person is standing between two traditional...
95,yaar.elh,1003,NaN,Dutch,Insufficient evidence to determine nationality.,The detected language is Dutch (71% confidence...
101,seventeen17_official,1022,NaN,South Korea,Insufficient evidence to determine nationality,SEVENTEEN is a well-known South Korean boy ban...
131,preciousekeh6,1103,NaN,Nigerian,Insufficient information to determine nationality,The individual's skin tone and facial features...
185,peterts115a,1224,NaN,Irish,Insufficient information to determine national...,The text in the image explicitly states 'Kells...
211,jxj03l,1337,NaN,Netherlands,Insufficient evidence to determine nationality.,"Language detected is Dutch, strongly indicatin..."


## Genelde Korea ve Korean gibi aynı tahminler farklı stringler. Bu yüzden tüm verinin yazımı

In [6]:
df[:40]

,username,creator_id,LLM tahmin,VLLM tahmin,LLM reasoning,VLLM reasoning
4,javirod305,726,United States,American,"Bio mentions Miami and NYC, indicating U.S. re...",Bio states 'A Miami boy living in NYC' and use...
12,queenisemhe1,747,NaN,Nigerian,Insufficient evidence to determine nationality.,The couple is wearing traditional Nigerian att...
16,dleysafricanhairbraiding,757,United States,American,"Bio mentions Camden, New Jersey, USA","Address 'Camden Nj' (New Jersey, USA) in bio s..."
22,calintee38,776,Canada,Canadian,Bio explicitly states 'Canadian creator',Bio explicitly states 'Canadian creator'.
25,labellegrachi,779,France,French,Bio and post text in French,High confidence in French language detection (...
29,lucasranngel,797,Brazil,Brazil,Portuguese language and cultural context indic...,Portuguese language detected with 100% confide...
31,stephcfitness_,802,Wales,Welsh,Bio explicitly states 'Wales',"Bio explicitly states '32 | Wales', indicating..."
35,sherylrubio3,818,Venezuela,Venezuelan,Bio explicitly states 'Venezuelan in NYC',Bio explicitly states 'Venezuelan in NYC' and ...
38,skyrockfm,827,France,France,"Bio mentions Skyrock, a French radio station.","Skyrock is a well-known French radio station, ..."
39,makeupbyyllke,828,Canada,Canada,Bio flag and Canadian phone number,Bio explicitly includes Canadian flag (🇨🇦) and...


In [7]:
df[40:80]

,username,creator_id,LLM tahmin,VLLM tahmin,LLM reasoning,VLLM reasoning
102,chilllit_seoul,1023,South Korea,"Korea, Republic of","Bio and flags indicate residence in Seoul, Sou...",Bio explicitly states 'Based in seoulcity 🇰🇷' ...
103,isabell.afro,1025,Denmark,Denmark,"Bio clearly states Danish dancer, overriding o...",Bio explicitly states 'Danish dancer🇩🇰' with h...
104,natiuv2,1033,Venezuela,Venezuela,Bio flag and content in Spanish indicate Venez...,"Bio explicitly flags 'Venezuela, Bolivarian Re..."
105,jorgejavier_diaz,1035,Spain,Spain,"Bio explicitly states Spain, overriding other ...",Bio explicitly states '📍Spain 🇪🇸' with the Spa...
106,yvatuspam,1038,France,NaN,"Bio and posts in French, no other flags.",No clear nationality indicators in text or ima...
108,thebridalcompanyofficial,1046,Australia,NaN,"Mention of Gold Coast, Australia in bio and posts",The profile image and text are in English and ...
110,fleekedbymitch,1056,United Kingdom,United Kingdom,Bio and location flag indicate UK residence,"Bio explicitly states 'Located in Nottingham, ..."
113,soundtiss,1069,Thailand,Thailand,Posts contain Thai language and cultural context,"Thai language in posts (e.g., 'ตอบกลับ', 'ครั้..."
114,dirtycashapparel,1070,Philippines,Philippines,Bio flag and emoji indicate Philippines,Bio explicitly contains the Philippine flag em...
119,sydsmithtattoo,1077,United States,NaN,"Bio mentions Scottsdale, AZ, indicating U.S. l...",No visual or textual indicators of nationality...


In [8]:
df[80:]

,username,creator_id,LLM tahmin,VLLM tahmin,LLM reasoning,VLLM reasoning
207,bronxsistas,1326,Germany,Germany,Bio and hashtags indicate Germany,"Bio explicitly flags Germany, uses German lang..."
208,kailani.fonseca,1333,Angola,Angola,Bio flag and Portuguese hashtags indicate Angola,Bio explicitly flags 'Angola' and includes the...
209,miahhcarter,1335,United Kingdom,United Kingdom,Email domain indicates UK management company,Email domain 'eminencemanagement.co.uk' indica...
211,jxj03l,1337,NaN,Netherlands,Insufficient evidence to determine nationality.,"Language detected is Dutch, strongly indicatin..."
213,juliusburphy,1355,United Kingdom,United Kingdom,Bio flag and content suggest UK origin,Bio explicitly includes the UK flag emoji (🇬🇧)...
215,myla.rnd,1358,France,French,"Text in French, no contradicting evidence.",The French phrase 'je voulais voir s’il regard...
216,alecotm,1359,France,French,"Bio explicitly states Paris, France.","Bio explicitly states 'Paris, France', and the..."


## Son olarak LLM'nin tahmin edip VLLM'nin tahmin edemediği

In [9]:
a = df[df["LLM tahmin"].notna() & df["VLLM tahmin"].isna()].shape[0]

print(f"VLLM modelinin tahmin edemeyip LLM Modelinin tahmin edebildiği toplam değer: {a}")
df[df["LLM tahmin"].notna() & df["VLLM tahmin"].isna()]

VLLM modelinin tahmin edemeyip LLM Modelinin tahmin edebildiği toplam değer: 12


,username,creator_id,LLM tahmin,VLLM tahmin,LLM reasoning,VLLM reasoning
46,voteforknope,850,United States,NaN,Bio mentions living in LA,No clear visual or textual indicators of natio...
60,livbeachlv,874,United States,NaN,"Mentions Fontainebleau Las Vegas, located in t...",The image and text refer to a venue (LIV Beach...
106,yvatuspam,1038,France,NaN,"Bio and posts in French, no other flags.",No clear nationality indicators in text or ima...
108,thebridalcompanyofficial,1046,Australia,NaN,"Mention of Gold Coast, Australia in bio and posts",The profile image and text are in English and ...
119,sydsmithtattoo,1077,United States,NaN,"Bio mentions Scottsdale, AZ, indicating U.S. l...",No visual or textual indicators of nationality...
126,user296724867146,1096,Brazil,NaN,Text 'Minnhe feira sexta' suggests Brazilian P...,Profile image is a generic placeholder with no...
142,akeishastuidoglam,1121,Jamaica,NaN,Linden is a town in Jamaica,No clear textual or visual indicators of natio...
155,theredchickz,1149,United States,NaN,"Bio mentions multiple US cities, posts in English",No visual or textual indicators of nationality...
156,agusflei,1154,Spain,NaN,Spanish language in bio and post,Insufficient visual or textual evidence to det...
164,arlua.r,1171,Spain,NaN,"Uses Spanish in bio and posts, mentions 'maqui...",No clear visual or textual indicators of natio...
